In [ ]:
import tqdm
import json

from utils import requestLLM
from LeaderAgent import Leader
from Project import Project
from MemberAgent import Member
from evaluation.trip_prompt import evaluation_metric
from evaluation.trip_prompt import new_travel_train

# 创建一个空列表来存储读取的字典数据
tasks = []

# 打开 JSONL 文件以读取模式
with open("../data/travel_data.jsonl", 'r') as jsonl_file:
    # 逐行读取文件内容
    for line in jsonl_file:
        # 解析每行的 JSON 对象并添加到列表中
        item = json.loads(line)
        tasks.append(item)

In [ ]:
def task2str(task):
    string = ""
    for k, v in task.items():
        string += f"{k}:"
        if type(v) == list:
            string += f"{', '.join(v)}\n"
        elif type(v) == dict:
            string += f"adults:{v['adults']}, children:{v['children']}\n"
        else:
            string += f"{v}\n"
    return string

In [ ]:


file_path = 'res_travel_test2.json'  # JSON文件路径
for num, total_task in enumerate(tasks):
    experiences = json.load(open('experience.json', 'r'))
    experience_leader = ""
    experience_member = ""
    # experience = experiences['experience'][-1]
    for experience in experiences['experience']:
        experience_leader += "Task\n" + experience['task'] + "\nLeader\n" + experience['Leader']
        experience_member += "Task\n" + experience['task'] + "\nMember\n" + '\n'.join(experience['Member'])
    total_task = task2str(total_task)
    print(total_task)
    leader = Leader("Leader", total_task)

    # leader.excerpt_experience(experience_leader)

    leader.sum_up_experience(experience_leader)

    main_goals, member_tasks, guidance = leader.decompose_task()
    project = Project(total_task, main_goals, guidance)
    members = []
    for role, task in member_tasks.items():
        print(f"{role}:{task}")
        members.append(Member(role, task, total_task))

    for member in members:
        member.summarize_global_experience(project, experience_member)

    res = {}
    res_pre = ""

    print("先完成一次工作")
    # 现在要流水线完成工作
    for member in members:
        response = member.complete_task(project,
                                        "\nThis is the work of the previous people, completing one's own work based on their work\n" + res_pre)
        res_pre += "\n\n" + response
        res[member.role] = response
        print(member.role, "complete")

    res_pre += "Role: Leader\n\n" + leader.complete_task(res_pre)

    print("开始循环，先讨论，再生成答案")

    local_experience = {}
    for member in members:
        local_experience[member.role] = ""
    for i in range(2):
        print("meeting")
        for member in members:
            member.meeting(res_pre)
            print(member.role, "meeting")

        leader.meeting(res_pre)

        print("根据feedback重做")
        # 提取feedback内容
        feedback_dict = {
            "Peer Score and Feedback": {},
            "Team Member Score and Evaluation": {}
        }
        for member in members:
            for heading, feedback in member.feedback.items():
                if heading != "Self-Evaluation Score and Feedback":
                    feedback_dict[heading][member.role] = feedback
        for heading, feedback in leader.feedback_dict.items():
            if heading != "Self-Evaluation Score and Feedback":
                feedback_dict[heading]["leader"] = feedback
        res_pre = ""
        print("会后总结")
        local_experience_str = "\n\n".join([f"{role}:{exp}" for role, exp in local_experience.items()])
        for member in members:
            feedback_received = {}
            for heading, feedbacks in feedback_dict.items():
                feedback_received[heading] = "\n".join(
                    [feedback for role, feedback in feedbacks.items() if role != member.role])
            feedback_received_str = "\n".join(
                [heading + ":\n" + feedback for heading, feedback in feedback_received.items()])
            local_experience[member.role] = member.summarize_local_experience(
                "\nThis is feedback and experience from others, based on their own improvement results\n" + feedback_received_str,
                local_experience_str)
            print(member.role, "local_exp")

        # print("使用经验、反馈重新生成")
        local_experience_str = "\n\n".join([exp for role, exp in local_experience.items()])
        # print(local_experience_str)
        for member in members:
            feedback_received = {}
            for heading, feedbacks in feedback_dict.items():
                feedback_received[heading] = "\n".join(
                    [feedback for role, feedback in feedbacks.items() if role != member.role])
            feedback_received_str = "\n".join(
                [heading + ":\n" + feedback for heading, feedback in feedback_received.items()])
            res_pre += "\n\n" + member.complete_task(project,
                                                     "\nThis is the work of the previous people, completing one's own work based on their work\n" + res_pre,
                                                     "\nThis is feedback from others, based on their own improvement results\n" + feedback_received_str,
                                                     "\nThese are some reference work experiences.\n" + local_experience_str)
            print(member.role, " completed")
            # )

        subtask_res = {}
        for member in members:
            subtask_res[member.role] = member.response
        subtask_res_str = "\n".join([role + '\n' + response for role, response in subtask_res.items()])
        con = leader.complete_task(subtask_res_str)
        res_pre += "Leader:\n\n" + con

    # from evaluation.evaluate import evaluate
    message = new_travel_train.format(
        total_task=total_task,
        Travel_Plan=leader.final_res
    )
    evaluation = requestLLM(message, "").content

    print(evaluation)

    # 打开文件进行写入
    with open(f"log/newTravel{num}_leader-test.txt", 'w', encoding='utf-8') as file:
        # 遍历列表并写入
        for string in leader.history:
            file.write(string + "\n")
    for ids, member in enumerate(members):
        with open(f"log/newTravel{num}_role{ids}-test.txt", 'w', encoding='utf-8') as file:
            for string in member.history:
                file.write(string + "\n")
    try:
        try:
            # 尝试打开已存在的JSON文件以读取和写入 ('r+' 表示读取和写入)
            with open(file_path, 'r+') as file:
                data = json.load(file)  # 读取现有数据
        except FileNotFoundError:
            # 如果文件不存在，创建一个新的空JSON数据结构
            data = {"res": []}

        # 添加新数据到数据结构中
        # data["res"].append({"task": total_task, "final_plan": leader.final_res, "evaluation": evaluation})
        data["res"].append({"task": total_task, "final_plan": leader.final_res})

        # 将数据写回文件
        with open(file_path, 'w') as file:
            json.dump(data, file, indent=4)  # indent参数可选，用于美化JSON格式

        print(f'task:{total_task} has completed')
    except Exception as e:
        print('Error for json:', str(e))
    # except Exception as e:
    #     print("============================")
    #     print(num)
    #     print("============================")
    # break